**1 задание**

В наличии данные по карточным транзакциям клиентов за 3 месяца. 
**Задача:**

Выделить клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев.

Упрощенный пример поиска транзакций по заданной теме на примере транзакций в аптеках:
Мы ищем в столбце merchant_name наименования с ключевым словом “аптека” и известные сети (Ригла, Живика, Самсон-Фарма и др.), дополнительно ограничивая результат мсс, соответствующим аптекам (5912 и 5122). В результате получаем набор транзакций в аптеках.


# **0. Предобработка данных**

Импортируем нужные библиотеки:

In [ ]:
import pandas as pd
import datetime

Файл с данными и с кодами MCC лежат в google drive. Будем читать их сразу от туда.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Файл с данными по кодам MCC был скачен по ссылке из описания данных.

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/transactions_small.csv', sep=';')
mcc_all = pd.read_csv('/gdrive/MyDrive/mcc_codes.csv')

In [ ]:
df.head(1)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.0,810,5999,i-bank ┴шыaщэ >moscow ru,moscow


Описание столбцов данных в карточных транзакциях:
* 'customer_id' - идентификатор клиента
* 'card_id' - идентификатор карты клиента
* 'tr_datetime' - дата совершения транзакции
* 'tr_normalized_amount' - сумма транзакции в рублях
* 'currency' - валюта транзакции
* 'mcc' - мсс код транзакции (что такое мсс и поиск значения можно посмотреть тут https://mcc-codes.ru/)
* 'merchant_name' - наименование торговой точки, где была произведена транзакция
* 'merchant_city' - город, в котором была произведена транзакция

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623956 entries, 0 to 623955
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customer_id           623956 non-null  object 
 1   card_id               623956 non-null  object 
 2   tr_datetime           623956 non-null  object 
 3   tr_normalized_amount  623956 non-null  float64
 4   currency              623956 non-null  int64  
 5   mcc                   623956 non-null  int64  
 6   merchant_name         623956 non-null  object 
 7   merchant_city         623953 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 38.1+ MB


Из общего описания данных видно, что пропусков нет. Заменим формат данных у даты. 

In [ ]:
df['tr_datetime'] = pd.to_datetime(df['tr_datetime'])

Отдельно добавим столбцы для года и месяца.

In [ ]:
df['tr_year'] = df['tr_datetime'].dt.year
df['tr_month'] = df['tr_datetime'].dt.month

Какие года и месяца в данном датасете?

In [ ]:
df['tr_year'].unique()

array([2019])

In [ ]:
df['tr_month'].unique()

array([ 9, 10,  8])

Так как у нас данные только для одного года, то повторов среди месяцев нет. Значит можно строить суммарные покупки по месяцам без учета года. Также у нас данные только за три месяца.

# **1. Выборка данных по покупкам кофе и кафетериям.**

Разделим поставленную задачу на две части:

1. Найдем коды MCC, которые подходят под описание покупки кофе или посещение кофейни.
2. Выделим из названия точек "merchant_name " те, где встречается кофе или кафе.

# **1.1 Выведем варианты расшифровки кода MCC подходящие под продажу кофе или кофейни.**

In [ ]:
list_mcc = [ 5499, 5541, 5812, 5813, 5814, 7832]

Для этого создадим фильтр для файла с кодами мсс

In [ ]:
mcc_filter_short = mcc_all['MCC'].isin(final_list_mcc)

Так как мы хотим вывести весь текст, то увеличим число символов.

In [ ]:
pd.options.display.max_colwidth = 500

In [ ]:
mcc_all[mcc_filter_short]

,MCC,Название,Описание
813,5499,Различные продовольственные магазины - нигде более не классифицированные,"Торговые точки, продающие продукты, не классифицированные в других категориях. Сюда относятся: специализированные продовольственные рынки, магазины диетических продуктов, деликатесов, домашней птицы, кофейни, овощные и фруктовые рынки, а также магазины мороженого, йогуртов и полуфабрикатов и небольшие магазины у дома. \nДля магазинов, которые также продают автомобильный бензин, используется MCC 5541. \nДля точек, которые в основном продают мясо и морепродукты, используется MCC 5422."
819,5541,Заправочные станции (с вспомогательными услугами или без),"Торговые точки, которые продают топливо для потребительского использования и могут или не могут также иметь на территории магазин, автомойку или авторемонтную мастерскую. Этот MCC включает станции техобслуживания, расположенных в гавани, у которых есть отдельное торговое соглашение от торгового терминала.\nДля транзакций, совершаемых на автоматических заправочных станциях, используется MCC 5542."
852,5812,"Места общественного питания, рестораны","Торговые точки, занимающиеся приготовлением еды и напитков для немедленного потребления, обычно на заказ. Могут оказывать услуги по обслуживанию столиков официантами. Места общественного питания включают в себя: кафе, кафетерии, грили, кофейни, закусочные, охлаждаемые прилавки для продажи мороженого и напитков, пиццерии, столовые, магазины деликатесов, бистро.\n\nДля точек, продающих еду быстрого приготовления, используется МСС 5814. Для точек, торгующих алкогольными напитками на заказ, исп..."
853,5813,"Бары, коктейль-бары, дискотеки, ночные клубы и таверны – места продажи алкогольных напитков","Торговые точки, продающие алкогольные напитки, такие как вино, пиво, эль, смешанные напитки и другие ликеры и напитки для потребления на заказ. Места продажи алкогольных напитков включают в себя: бары, пивные, пабы, коктейль-бары, дискотеки, ночные клубы, таверны и винные бары."
854,5814,Фастфуд,"Торговые точки, продающие готовую еду и напитки для немедленного потребления, как на заказ, так и упакованную на вынос. Такие рестораны могут использовать мак-драйв для приема и выдачи заказов и обычно не предоставляют услуги по обслуживанию столиков официантами и не берут чаевые. Эти торговые точки обычно не продают алкогольные напитки."
995,7832,Кинотеатры,"Точки, которые управляют кинотеатрами, в том числе под открытым небом (drive-in). Такие продавцы продают билеты и напитки и могут предлагать или не предлагать предварительное бронирование билетов по телефону."


Из всего этого списка оставим только наиболее подходящие коды, связанные с ресторанами, кофейнями и фастфудом.

Создадим фильтр для общего датасета данных:

In [ ]:
final_list_mcc_2 = [5812, 5814]
df_filter_short_2 = df['mcc'].isin(final_list_mcc_2)
data_mcc = df[df_filter_short_2]

Таким образом датасет data_mcc содержит в себе все транзакции связанные с кодами mcc 5812 и 5814.

# **1.2 Выделим merchant_name, где встречается сочетание кофе или кафе.**

Для поиска таких элементов создадим фильтр для сочетаний: kofe, kafe, koffe, coffe, cafe.

То есть учтем, что в названии могут использоваться слова кофе или кафе с опечатками.

In [ ]:
data_coffee = df[(df['merchant_name'].str.contains('kafe')) | 
   (df['merchant_name'].str.contains('kofe'))|
   (df['merchant_name'].str.contains('coffe'))|
   (df['merchant_name'].str.contains('cofe')) |
   (df['merchant_name'].str.contains('koffe'))]

data_coffee это данные по продажам кофе или в кофейнях. Однако здесь мы не учитываем популярные рестораны и фастфуды, где также можно купить кофе. Чтоб объединить данные по выборке по merchant_name и кодам мсс, сделаем объединение полученных датасетов. 

Для этого надо из полученного дата сета вырезать данные по коду мсс, во избежание повторов.

In [ ]:
data_temp = data_coffee[~data_coffee['mcc'].isin([5812, 5814])]

In [ ]:
result_data = pd.concat([data_mcc, data_temp]).reset_index(drop=True)

Таким образом мы получили два дата сета:
1. data_coffee - фрейм, где учитываются только упоминание кофеен и кофе в названии merchant_name
2. result_data -  объединенный фрейм, где укчитываются и коды мсс и упоминание кофе в merchant_name.

# **2. Выделение клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев.**

Создадим функцию для вывода списка customer_id тех, кто покупали кофе/ посещали кофейни больше двух раз 2 из 3 месяцев.

В самой функции мы введем дополнительный столбец данных 'id_coffee', который будет равен 1, если покупок было больше двух в месяц и 0, если только 1.

In [ ]:
def client_id(dataframe):
  table = dataframe.pivot_table(index=['customer_id', 'tr_month'], 
                                values='merchant_name', aggfunc='count'
                ).reset_index().sort_values(by='merchant_name', 
                                            ascending=False
                                            ).reset_index(drop=True)
  table['id_coffee'] = [1
                      if table.loc[el, 'merchant_name'] > 1
                      else 0
                      for el in range(len(table))]
  table_fin = table.pivot_table(index='customer_id', values='id_coffee', 
                              aggfunc='count').reset_index()
  list_id = list(table_fin[table_fin['id_coffee'] >1]['customer_id'])
  return list_id

In [ ]:
list_coffee = client_id(data_coffee)
list_all = client_id(result_data)

Таким образом можно сделать вывод, что:

In [ ]:
print("Количество клиентов, которые покупали кофе/посещали кофейни от 2 раз в \
месяц хотя бы 2 из 3 месяцев находится  между", 
len(list_coffee), "и", len(list_all))

Количество клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев находится  между 952 и 3400
